<a href="https://colab.research.google.com/github/jwb4335/nlp/blob/master/hw2_part1/hw2part1_johnbarry_20200120.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Homework 2 part 1
## CS 590.07
## John Barry





## Load in simlex999 data from github

In [0]:
import pandas as pd
from random import randint



In [0]:
url = 'https://raw.githubusercontent.com/jwb4335/nlp/master/hw2_part1/Sim_Lex999.txt'
simlex = pd.read_csv(url,sep = ':')
simlex.columns = ['word1','word2','SimLex999','POS1','POS2']



```
# This is formatted as code
```

Let's get three random rows

In [3]:
randints = [randint(0,len(simlex)), randint(0,len(simlex)),randint(0,len(simlex))]

simlex_rand = simlex.iloc[randints,:]
simlex_rand



,word1,word2,SimLex999,POS1,POS2
81,polite,proper,7.63,a,a
369,cloud,weather,4.87,n,n
163,way,manner,7.62,n,n


## **Universal sentence encoder** 





In [0]:
# Install the latest Tensorflow version.
!pip3 install --quiet "tensorflow>=1.7"
# Install TF-Hub.
!pip3 install --quiet tensorflow-hub
!pip3 install --quiet seaborn

In [5]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
import seaborn as sns

In [0]:
module_url = "https://tfhub.dev/google/universal-sentence-encoder/2" #@param ["https://tfhub.dev/google/universal-sentence-encoder/2", "https://tfhub.dev/google/universal-sentence-encoder-large/3"]

In [7]:
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(module_url)

# Compute a representation for each message, showing various lengths supported.
word = "breakfast"
messages = [word]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))
    print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

Message: breakfast
Embedding size: 512
Embedding: [-0.016404513269662857, -0.04483848437666893, -0.0037806909531354904, ...]



In [8]:
message_embedding_snippet

'-0.016404513269662857, -0.04483848437666893, -0.0037806909531354904'

In [0]:
word = "breakfast"
messages = [word]

# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as session:
  session.run([tf.global_variables_initializer(), tf.tables_initializer()])
  message_embeddings = session.run(embed(messages))

In [10]:
message_embeddings

array([[-1.64045133e-02, -4.48384844e-02, -3.78069095e-03,
         4.68277894e-02, -7.71008851e-03, -5.05571701e-02,
        -6.78127445e-03, -3.27876322e-02, -6.28904551e-02,
        -4.78958301e-02,  5.76800331e-02, -5.30807972e-02,
         2.26386786e-02,  2.00913250e-02, -1.91726331e-02,
         1.45138241e-02, -9.16321948e-03, -3.39176925e-03,
        -2.80498969e-03, -3.79767418e-02,  4.18685153e-02,
        -6.24692850e-02, -2.87507810e-02, -3.40368375e-02,
        -3.47199216e-02, -2.53357105e-02, -4.59449776e-02,
        -7.09898174e-02, -6.53066039e-02,  3.80705930e-02,
        -3.92757617e-02, -2.31163409e-02, -3.94491926e-02,
         1.65286213e-02,  3.73129509e-02, -5.67618310e-02,
        -3.04891113e-02,  5.97833432e-02, -2.56873239e-02,
        -6.43153442e-04,  5.93997464e-02,  5.53347953e-02,
         2.13192087e-02, -5.59708215e-02,  4.60403711e-02,
         2.85874866e-02,  3.25253457e-02, -7.22681209e-02,
         5.07617891e-02, -1.03512416e-02, -5.29682003e-0

# CBOW or word2index

Download the data

In [11]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/reviews.txt"
!cat reviews.txt | head

--2020-01-26 12:26:10--  https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 33678267 (32M) [text/plain]
Saving to: ‘reviews.txt’

reviews.txt         100%[===================>]  32.12M  40.9MB/s    in 0.8s    

2020-01-26 12:26:13 (40.9 MB/s) - ‘reviews.txt’ saved [33678267/33678267]

bromwell high is a cartoon comedy . it ran at the same time as some other programs about school life  such as  teachers  . my   years in the teaching profession lead me to believe that bromwell high  s satire is much closer to reality than is  teachers  . the scramble to survive financially  the insightful students who can see right through their pathetic teachers  pomp  the pettiness of the wh

In [12]:
!wget "https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/labels.txt"
!cat labels.txt | head -15

--2020-01-26 12:26:17--  https://raw.githubusercontent.com/udacity/deep-learning/master/sentiment-network/labels.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 225000 (220K) [text/plain]
Saving to: ‘labels.txt’

labels.txt          100%[===================>] 219.73K  --.-KB/s    in 0.05s   

2020-01-26 12:26:18 (4.34 MB/s) - ‘labels.txt’ saved [225000/225000]

positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive
negative
positive


Predicting Movie Reviews

In [0]:
import sys

f = open('reviews.txt')
raw_reviews = f.readlines()
f.close()

f = open('labels.txt')
raw_labels = f.readlines()
f.close()

tokens = list(map(lambda x:set(x.split(" ")),raw_reviews))

vocab = set()
for sent in tokens:
    for word in sent:
        if(len(word)>0):
            vocab.add(word)
vocab = list(vocab)

word2index = {}
for i,word in enumerate(vocab):
    word2index[word]=i

input_dataset = list()
for sent in tokens:
    sent_indices = list()
    for word in sent:
        try:
            sent_indices.append(word2index[word])
        except:
            ""
    input_dataset.append(list(set(sent_indices)))

target_dataset = list()
for label in raw_labels:
    if label == 'positive\n':
        target_dataset.append(1)
    else:
        target_dataset.append(0)

In [14]:
vocab[11]

'lowering'

In [15]:
vocab

['isco',
 'egos',
 'althou',
 'cocking',
 'precedes',
 'yours',
 'orgolini',
 'birnley',
 'gall',
 'warships',
 'banding',
 'lowering',
 'palatial',
 'placement',
 'sm',
 'extends',
 'tumors',
 'colour',
 'gillian',
 'simultaneous',
 'ettore',
 'bp',
 'brooked',
 'evolutions',
 'packaged',
 'champagne',
 'defile',
 'binev',
 'syllable',
 'apt',
 'tete',
 'howz',
 'gambling',
 'kieron',
 'bountiful',
 'misery',
 'kong',
 'osmond',
 'morsa',
 'hussar',
 'fjaestad',
 'indoctrination',
 'buckwheat',
 'convince',
 'costume',
 'momo',
 'stature',
 'sinisterness',
 'apprehend',
 'shah',
 'tenth',
 'decoding',
 'hatter',
 'unopposed',
 'simplicity',
 'infantilising',
 'portable',
 'hupfel',
 'discriminates',
 'shootout',
 'macs',
 'nikolaus',
 'concede',
 'citizens',
 'clarity',
 'ridicule',
 'tbh',
 'tpb',
 'surrender',
 'meted',
 'johannsson',
 'ctrlv',
 'against',
 'tommorrow',
 'forgery',
 'presidency',
 'ambivalence',
 'emergance',
 'clarrissa',
 'roldn',
 'purbbs',
 'iridescent',
 'simul

Initialize Weights

In [16]:
import numpy as np
np.random.seed(1)

def sigmoid(x):
    return 1/(1 + np.exp(-x))

alpha, iterations = (0.01, 2)
hidden_size = 100

weights_0_1 = 0.2*np.random.random((len(vocab),hidden_size)) - 0.1
weights_1_2 = 0.2*np.random.random((hidden_size,1)) - 0.1


print (weights_0_1)

[[-0.0165956   0.0440649  -0.09997713 ...  0.0147359  -0.09942593
   0.02342898]
 [-0.03467102  0.00541162  0.07718842 ... -0.05312758  0.02335567
   0.08980326]
 [ 0.09003522  0.01133064  0.08312127 ... -0.08407478  0.09656342
  -0.06367743]
 ...
 [-0.07656205 -0.09023205 -0.00978559 ... -0.0008865  -0.04352412
  -0.06427378]
 [-0.08444666 -0.07143386 -0.05731608 ... -0.06584719 -0.02770722
   0.09327983]
 [ 0.03695404 -0.05106593  0.02970002 ...  0.0043325   0.00801544
  -0.09468272]]


Run the Model

In [17]:


correct,total = (0,0)

for iter in range(iterations):
    
    # train on first 24,000
    for i in range(len(input_dataset)-1000):

        x,y = (input_dataset[i],target_dataset[i])
        layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0)) #embed + sigmoid
        layer_2 = sigmoid(np.dot(layer_1,weights_1_2)) # linear + softmax

        layer_2_delta = layer_2 - y # compare pred with truth
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) #backprop

        weights_0_1[x] -= layer_1_delta * alpha
        weights_1_2 -= np.outer(layer_1,layer_2_delta) * alpha

        if(np.abs(layer_2_delta) < 0.5):
            correct += 1
        total += 1
        if(i % 10 == 9):
            progress = str(i/float(len(input_dataset)))
            sys.stdout.write('\rIter:'+str(iter)\
                             +' Progress:'+progress[2:4]\
                             +'.'+progress[4:6]\
                             +'% Training Accuracy:'\
                             + str(correct/float(total)) + '%')
    print()
    
correct,total = (0,0)
for i in range(len(input_dataset)-1000,len(input_dataset)):

    x = input_dataset[i]
    y = target_dataset[i]

    layer_1 = sigmoid(np.sum(weights_0_1[x],axis=0))
    layer_2 = sigmoid(np.dot(layer_1,weights_1_2))
    
    if(np.abs(layer_2 - y) < 0.5):
        correct += 1
    total += 1
print("Test Accuracy:" + str(correct / float(total)))




Iter:0 Progress:95.99% Training Accuracy:0.83425%
Iter:1 Progress:95.99% Training Accuracy:0.8672708333333333%
Test Accuracy:0.846


Let's get the vector for a word

In [18]:
target = "dentist"
target_index = word2index[target]
weights_0_1[target_index]

array([-0.05600449,  0.05351525,  0.0111625 , -0.00445785, -0.10448053,
        0.07612028, -0.02910379, -0.0426673 , -0.08854123, -0.07809131,
        0.05540542, -0.01029449,  0.05904023, -0.09228346,  0.00485652,
        0.09514934, -0.06512389, -0.01227349,  0.02710689,  0.00724793,
       -0.03945616,  0.01782332,  0.0054515 ,  0.02617963, -0.02392748,
       -0.04004179, -0.0546614 ,  0.07599398,  0.0693973 ,  0.01997399,
        0.03522681,  0.08207886, -0.0796594 ,  0.0536613 ,  0.02551287,
        0.006994  , -0.07112999,  0.01688216, -0.01792746, -0.06112864,
        0.096187  ,  0.06642197,  0.03872604,  0.01670518,  0.10853636,
        0.04583814,  0.00066247, -0.0646109 ,  0.01098651, -0.05712537,
       -0.09512438, -0.00547048,  0.01614729, -0.07497346,  0.02274946,
       -0.03246368,  0.00071955, -0.07128209,  0.08528273,  0.06477119,
       -0.04606829, -0.01490683, -0.04745424,  0.08532691, -0.00165276,
        0.04494015, -0.00328174,  0.00305977, -0.00233395, -0.08

# Word2vec

In [19]:
!wget http://mattmahoney.net/dc/text8.zip -O text8.zip

--2020-01-26 12:26:37--  http://mattmahoney.net/dc/text8.zip
Resolving mattmahoney.net (mattmahoney.net)... 67.195.197.75
Connecting to mattmahoney.net (mattmahoney.net)|67.195.197.75|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31344016 (30M) [application/zip]
Saving to: ‘text8.zip’

text8.zip           100%[===================>]  29.89M   329KB/s    in 94s     

2020-01-26 12:28:11 (326 KB/s) - ‘text8.zip’ saved [31344016/31344016]



In [20]:
!unzip text8.zip

Archive:  text8.zip
  inflating: text8                   


In [21]:
!pip install gensim  

In [0]:
from gensim.models import word2vec

In [23]:
# https://github.com/dav/word2vec/blob/master/scripts/create-text8-vector-data.sh#L16
model = word2vec.Word2Vec(corpus_file='text8',size=100, window=5, min_count=1, workers=4)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Let's get the vector for a word

In [24]:
vector = model.wv['computer']
print(vector)

[ 0.12310577  0.05132691 -0.00861285 -0.0245771  -0.06153418  0.05641732
 -0.10804733 -0.03420261  0.10759732  0.02303753  0.01456071 -0.09585664
 -0.02623927  0.03934142  0.03090817  0.05634154  0.03749996 -0.04225422
 -0.01544135 -0.0398049   0.05176081 -0.0325308  -0.03991497 -0.00281919
 -0.03612226  0.10927431 -0.00349872 -0.01170234 -0.01116402 -0.04540886
  0.00315461  0.0833443   0.16819715  0.00891368  0.01751684  0.0115212
 -0.01976072 -0.01405599 -0.03039536 -0.04073559  0.02864988  0.14347737
  0.05116322  0.00727141  0.02885923  0.05884755  0.06352183  0.04836405
  0.03400828 -0.00206001 -0.03121194 -0.08110482 -0.03721883 -0.0023942
 -0.10083164  0.08444286  0.04763588 -0.01143563  0.07705823 -0.07866716
 -0.02662024  0.03243002  0.0296375   0.08581599 -0.01952261 -0.04867808
  0.05205264  0.03197421 -0.0435137  -0.03809602  0.08544057 -0.13082847
 -0.04993257  0.0618914   0.06549787  0.00341216 -0.01780589 -0.00704754
 -0.19065215 -0.01431869  0.00382879  0.05427075  0.0

# Get the similarity scores of the three embeddings

In [0]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
pd.options.mode.chained_assignment = None  # default='warn'
for num in randints:
  word1 = simlex_rand.loc[simlex_rand.index == num]['word1'].values[0]
  word2 = simlex_rand.loc[simlex_rand.index == num]['word2'].values[0]
  print('Embedding this pair from Simlex_999: "{}" and "{}"'.format(word1, word2))
  print('Running universal sentence encoder on "{}"'.format(word1))
  messages = [word1]
  # Reduce logging output.
  tf.logging.set_verbosity(tf.logging.ERROR)

  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(messages))

    for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
      print("Message: {}".format(messages[i]))
      print("Embedding size: {}".format(len(message_embedding)))
      message_embedding_snippet = ", ".join(
          (str(x) for x in message_embedding[:3]))
      print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

  vector1 = message_embedding
  vector1 = np.asarray(vector1)
  vector1 = vector1.reshape(1,512)

  print('Running universal sentence encoder on "{}"'.format(word2))
  messages = [word2]

  # Reduce logging output.
  tf.logging.set_verbosity(tf.logging.ERROR)

  with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    message_embeddings = session.run(embed(messages))

  for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
    print("Message: {}".format(messages[i]))
    print("Embedding size: {}".format(len(message_embedding)))
    message_embedding_snippet = ", ".join(
        (str(x) for x in message_embedding[:3]))

  vector2 = message_embedding
  vector2 = np.asarray(vector2)
  vector2 = vector2.reshape(1,512)

  cos_sim = cosine_similarity(vector1, vector2)
  print('Cosine similarity from Universal sentence encoder for {} and {} = {}'.format(word1,word2,cos_sim) )
  simlex_rand.loc[simlex_rand.index == num,'USE Cosine Sim'] = cos_sim

Embedding this pair from Simlex_999: "polite" and "proper"
Running universal sentence encoder on "polite"
Message: polite
Embedding size: 512
Embedding: [-0.03852936252951622, 0.0010572668397799134, -0.03292347118258476, ...]

Running universal sentence encoder on "proper"
Message: proper
Embedding size: 512
Cosine similarity from Universal sentence encoder for polite and proper = [[0.62162989]]
Embedding this pair from Simlex_999: "cloud" and "weather"
Running universal sentence encoder on "cloud"
Message: cloud
Embedding size: 512
Embedding: [0.02739393711090088, 0.05895265191793442, -0.016418352723121643, ...]

Running universal sentence encoder on "weather"
Message: weather
Embedding size: 512
Cosine similarity from Universal sentence encoder for cloud and weather = [[0.45271238]]
Embedding this pair from Simlex_999: "way" and "manner"
Running universal sentence encoder on "way"
Message: way
Embedding size: 512
Embedding: [-0.03555486351251602, 0.003041286952793598, -0.018496295437

Embedding method 2

In [27]:
for num in randints:
  word1 = simlex_rand.loc[simlex_rand.index == num]['word1'].values[0]
  word2 = simlex_rand.loc[simlex_rand.index == num]['word2'].values[0]
  print('running word2index on the pair {} and {}'.format(word1,word2))
  target_index1 = word2index[word1]
  target_index2 = word2index[word2]
  vector1 = weights_0_1[target_index1]
  vector1 = vector1.reshape(1,100)
  vector2 = weights_0_1[target_index2]
  vector2 = vector2.reshape(1,100)
  cos_sim = cosine_similarity(vector1, vector2)
  print('Cosine similarity from word2index for {} and {} = {}'.format(word1,word2,cos_sim))
  simlex_rand.loc[simlex_rand.index == num,'word2index Cosine Sim'] = cos_sim


running word2index on the pair polite and proper
Cosine similarity from word2index for polite and proper = [[0.01027398]]
running word2index on the pair cloud and weather
Cosine similarity from word2index for cloud and weather = [[0.00043138]]
running word2index on the pair way and manner
Cosine similarity from word2index for way and manner = [[0.08176885]]


Embedding method word2vec



In [28]:
for num in randints:
  word1 = simlex_rand.loc[simlex_rand.index == num]['word1'].values[0]
  word2 = simlex_rand.loc[simlex_rand.index == num]['word2'].values[0]
  print('running word2vec on the pair {} and {}'.format(word1,word2))
  vector1 = model.wv[word1]
  vector1 = vector1.reshape(1,100)
  vector2 = model.wv[word2]
  vector2 = vector2.reshape(1,100)
  cos_sim = cosine_similarity(vector1, vector2)
  print('Cosine similarity from word2vec for {} and {} = {}'.format(word1,word2,cos_sim))
  simlex_rand.loc[simlex_rand.index == num,'word2vec Cosine Sim'] = cos_sim



running word2vec on the pair polite and proper
Cosine similarity from word2vec for polite and proper = [[-0.05817803]]
running word2vec on the pair cloud and weather
Cosine similarity from word2vec for cloud and weather = [[0.11023503]]
running word2vec on the pair way and manner
Cosine similarity from word2vec for way and manner = [[0.8605586]]


In [29]:
simlex_rand_out = simlex_rand[['word1','word2','SimLex999', 'USE Cosine Sim',
       'word2index Cosine Sim', 'word2vec Cosine Sim']]
simlex_rand_out.columns = ['word1','word2','SimLex999', 'USE',
       'CBOW', 'word2vec']
simlex_rand_out = round(simlex_rand_out,4)
print('The cosine similarities from the 3 word embedding methods can be seen below.\nThe similarity scores are not close to the Simlex999 scores')
print(simlex_rand_out)

The cosine similarities from the 3 word embedding methods can be seen below.
The similarity scores are not close to the Simlex999 scores
      word1    word2  SimLex999     USE    CBOW  word2vec
81   polite   proper       7.63  0.6216  0.0103   -0.0582
369   cloud  weather       4.87  0.4527  0.0004    0.1102
163     way   manner       7.62  0.7624  0.0818    0.8606


# Part 2, generate three .txt files with similarity scores of all words ending in 4, using my three embedded models.

In [30]:
## Unique id is jwb74, but python starts on zero, so I need to minus 1 to get the right words
unique_id = 4 
index = np.arange(4-1,1000,10)
simlex_out = simlex.iloc[index,:].reset_index(drop=True)[['word1','word2','SimLex999']]
simlex_out.columns = ['word1','word2','Sim_lex_Similarity']
simlex_out

,word1,word2,Sim_lex_Similarity
0,happy,cheerful,9.55
1,bad,terrible,7.78
2,dumb,foolish,6.67
3,weird,normal,0.72
4,nice,cruel,0.67
...,...,...,...
95,try,think,2.62
96,leave,appear,0.97
97,put,hang,3.00
98,say,verify,4.90


## Embedding method 1

In [0]:
import copy
simlex_out_USE = copy.deepcopy(simlex_out)
for num in np.arange(0,100,1):
  print('word pair {}'.format(num))
  word1 = simlex_out.loc[simlex_out.index == num]['word1'].values[0]
  word2 = simlex_out.loc[simlex_out.index == num]['word2'].values[0]
  try:
    print('Embedding this pair from Simlex_999: "{}" and "{}"'.format(word1, word2))
    #print('Running universal sentence encoder on "{}"'.format(word1))
    messages = [word1]
    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)

    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(messages))

      for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
        #print("Message: {}".format(messages[i]))
        #print("Embedding size: {}".format(len(message_embedding)))
        message_embedding_snippet = ", ".join(
            (str(x) for x in message_embedding[:3]))
        #print("Embedding: [{}, ...]\n".format(message_embedding_snippet))

    vector1 = message_embedding
    vector1 = np.asarray(vector1)
    vector1 = vector1.reshape(1,512)

    #print('Running universal sentence encoder on "{}"'.format(word2))
    messages = [word2]

    # Reduce logging output.
    tf.logging.set_verbosity(tf.logging.ERROR)

    with tf.Session() as session:
      session.run([tf.global_variables_initializer(), tf.tables_initializer()])
      message_embeddings = session.run(embed(messages))

    for i, message_embedding in enumerate(np.array(message_embeddings).tolist()):
      #print("Message: {}".format(messages[i]))
      #print("Embedding size: {}".format(len(message_embedding)))
      message_embedding_snippet = ", ".join(
          (str(x) for x in message_embedding[:3]))

    vector2 = message_embedding
    vector2 = np.asarray(vector2)
    vector2 = vector2.reshape(1,512)

    cos_sim = cosine_similarity(vector1, vector2)
    #print('Cosine similarity from Universal sentence encoder for {} and {} = {}'.format(word1,word2,cos_sim) )
    simlex_out_USE.loc[simlex_out_USE.index == num,'your_Similarity'] = cos_sim
  except:
    print('skipping the pair {} and {}'.format(word1,word2))


## Embedding method 2

In [0]:
simlex_out_CBOW = copy.deepcopy(simlex_out)
for num in np.arange(0,100,1):
  word1 = simlex_out.loc[simlex_out.index == num]['word1'].values[0]
  word2 = simlex_out.loc[simlex_out.index == num]['word2'].values[0]
  #print('running word2index on the pair {} and {}'.format(word1,word2))
  try:
    target_index1 = word2index[word1]
    target_index2 = word2index[word2]
    vector1 = weights_0_1[target_index1]
    vector1 = vector1.reshape(1,100)
    vector2 = weights_0_1[target_index2]
    vector2 = vector2.reshape(1,100)
    cos_sim = cosine_similarity(vector1, vector2)
    #print('Cosine similarity from word2index for {} and {} = {}'.format(word1,word2,cos_sim))
    simlex_out_CBOW.loc[simlex_out_CBOW.index == num,'your_Similarity'] = cos_sim
  except:
    #print('skipping the pair {} and {}'.format(word1,word2))


## Embedding method 3

In [0]:
simlex_out_word2vec = copy.deepcopy(simlex_out)
for num in np.arange(0,100,1):
  word1 = simlex_out.loc[simlex_out.index == num]['word1'].values[0]
  word2 = simlex_out.loc[simlex_out.index == num]['word2'].values[0]
  try:
    #print('running word2vec on the pair {} and {}'.format(word1,word2))
    vector1 = model.wv[word1]
    vector1 = vector1.reshape(1,100)
    vector2 = model.wv[word2]
    vector2 = vector2.reshape(1,100)
    cos_sim = cosine_similarity(vector1, vector2)
    print('Cosine similarity from word2vec for {} and {} = {}'.format(word1,word2,cos_sim))
    simlex_out_word2vec.loc[simlex_out_word2vec.index == num,'your_Similarity'] = cos_sim
  except:
    #print('skipping the pair {} and {}'.format(word1,word2))



## Output files to csv

In [0]:
from google.colab import files
simlex_out_USE.to_csv('universal_sentence_encoder.csv',sep='\t')
simlex_out_CBOW.to_csv('CBOW.csv', sep='\t')
simlex_out_word2vec.to_csv('word2vec.csv',sep='\t')
files.download('universal_sentence_encoder.csv')
files.download('CBOW.csv')
files.download('word2vec.csv')
